In [1]:
from os.path import expanduser, join, abspath

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.functions import date_format
from datetime import datetime
import pyspark.sql.types as T
import pyspark.sql.functions as F
# Initialize Spark Session
spark = SparkSession \
    .builder \
    .appName("featuresdata processing") \
    .getOrCreate()

24/08/30 12:21:27 WARN Utils: Your hostname, techie-data-System-Product-Name resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface enp6s0)
24/08/30 12:21:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 12:21:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
stores = spark.read.csv('/home/techie-data/RetailStream_ Comprehensive Data Integration and Analytics Pipeline/Store_Dataset/*.csv', header=True, inferSchema=True)
stores.show()
stores.count()

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
|    3|   B| 37392|
|    4|   A|205863|
|    5|   B| 34875|
|    6|   A|202505|
|    7|   B| 70713|
|    8|   A|155078|
|    9|   B|125833|
|   10|   B|126512|
|   11|   A|207499|
|   12|   B|112238|
|   13|   A|219622|
|   14|   A|200898|
|   15|   B|123737|
|   16|   B| 57197|
|   17|   B| 93188|
|   18|   B|120653|
|   19|   A|203819|
|   20|   A|203742|
+-----+----+------+
only showing top 20 rows



45

In [3]:
sales = spark.read.csv('/home/techie-data/RetailStream_ Comprehensive Data Integration and Analytics Pipeline/Sales_Dataset/*.csv', header=True, inferSchema=True)
sales.show()
sales.count()

+-----+----+----------+------------+---------+
|Store|Dept|      Date|Weekly_Sales|IsHoliday|
+-----+----+----------+------------+---------+
|   14|  90|19/08/2011|   107057.98|    false|
|   14|  90|26/08/2011|    138231.7|    false|
|   14|  90|02/09/2011|    90563.66|    false|
|   14|  90|09/09/2011|   128470.61|     true|
|   14|  90|16/09/2011|   111020.47|    false|
|   14|  90|23/09/2011|   110587.89|    false|
|   14|  90|30/09/2011|   105622.09|    false|
|   14|  90|07/10/2011|   126129.76|    false|
|   14|  90|14/10/2011|   108236.99|    false|
|   14|  90|21/10/2011|   106148.35|    false|
|   14|  90|28/10/2011|   108059.12|    false|
|   14|  90|04/11/2011|   112685.75|    false|
|   14|  90|11/11/2011|    108969.6|    false|
|   14|  90|18/11/2011|   107175.51|    false|
|   14|  90|25/11/2011|   125005.24|     true|
|   14|  90|02/12/2011|    99589.33|    false|
|   14|  90|09/12/2011|   119985.86|    false|
|   14|  90|16/12/2011|   115607.91|    false|
|   14|  90|2

421570

In [4]:
features = spark.read.csv('/home/techie-data/RetailStream_ Comprehensive Data Integration and Analytics Pipeline/Features_data_set/*.csv', header=True, inferSchema=True)
features.show()
features.count()

+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|Store|      Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|IsHoliday|
+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|   17|14/10/2011|      47.69|     3.567|     NULL|     NULL|     NULL|     NULL|     NULL|129.7706452|       6.617|    false|
|   17|21/10/2011|      47.27|     3.579|     NULL|     NULL|     NULL|     NULL|     NULL|129.7821613|       6.617|    false|
|   17|28/10/2011|      41.89|     3.567|     NULL|     NULL|     NULL|     NULL|     NULL|129.7936774|       6.617|    false|
|   17|04/11/2011|      37.52|     3.538|     NULL|     NULL|     NULL|     NULL|     NULL|129.8051935|       6.617|    false|
|   17|11/11/2011|      27.61|     3.513|   3935.0|  4172.99|   225.67|  2515.61|  3009.79|129.8167097|       6

8190

In [5]:
Retail_frame=features.join(stores,["store"]) \
     .join(sales,[features["date"] == sales["date"],features["store"] == sales["store"]]).drop(sales.Store,sales.Date,sales.IsHoliday,stores.Store).fillna(0)
def user_defined_timestamp(date_col):
    _date = datetime.strptime(date_col, '%d/%m/%Y')
    return _date.strftime('%Y-%m-%d')

user_defined_timestamp_udf = F.udf(user_defined_timestamp, T.StringType())
Retail_frame = Retail_frame.withColumn('Date_formatted', user_defined_timestamp_udf('Date')).drop("Date")
Retail_frame.show()

+-----+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+----+------+----+------------+--------------+
|Store|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|IsHoliday|Type|  Size|Dept|Weekly_Sales|Date_formatted|
+-----+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+----+------+----+------------+--------------+
|   14|      54.47|       4.0|  3214.56|      0.0|    20.28|   822.73|   7207.7|192.3272654|       8.667|    false|   A|200898|  90|    90327.67|    2012-10-12|
|   14|      54.47|       4.0|  3214.56|      0.0|    20.28|   822.73|   7207.7|192.3272654|       8.667|    false|   A|200898|  91|    76052.46|    2012-10-12|
|   14|      54.47|       4.0|  3214.56|      0.0|    20.28|   822.73|   7207.7|192.3272654|       8.667|    false|   A|200898|  92|   156174.21|    2012-10-12|
|   14|      54.47|       4.0|  32

In [37]:
Retail_frame.createOrReplaceTempView("retail_data")
spark.sql("select sum(weekly_sales) from retail_data where date_formatted ='2010-02-05' and store =1").show()

+------------------+
| sum(weekly_sales)|
+------------------+
|1643690.9000000001|
+------------------+



Customer Visit Analysis:

In [7]:
April_sales_df = Retail_frame.drop("Temperature","Fuel_Price","MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5")
April_sales_df.createOrReplaceTempView("april_sales")
April_sales = spark.sql("select Store, Dept,Date_formatted,CPI,Weekly_Sales from april_sales where month(Date_formatted) = 4 and Type ='B'")
April_sales.createOrReplaceTempView("april_sales")
customer_april = spark.sql("select cast(avg (Total_customers) as int) as  Average_customers from (select Store,Date_formatted,round((sum(Weekly_Sales)/CPI),2) as Total_customers from april_sales group by Date_formatted,store,CPI order by store asc)").show()

+-----------------+
|Average_customers|
+-----------------+
|             5362|
+-----------------+



Holiday Sales Analysis:

In [8]:
Retail_frame.createOrReplaceTempView("holiday_sales")
holiday_sales = spark.sql("select Type,round(avg(Weekly_Sales),2) as Weekly_Sales from holiday_sales where IsHoliday ='true' group by Type order by Weekly_Sales desc")
holiday_sales.show()

+----+------------+
|Type|Weekly_Sales|
+----+------------+
|   A|    21297.52|
|   B|    13346.16|
|   C|     9532.96|
+----+------------+



Leap Year Sales Analysis:

In [16]:
Retail_frame.createOrReplaceTempView("leap_yr_sales")
leap_yr_sales = spark.sql("select Store,round(avg(Weekly_Sales),2) as Weekly_Sales from leap_yr_sales where (year(Date_formatted)%4=0 AND year(Date_formatted)%100!=0) OR year(Date_formatted)%400=0 group by Store order by Weekly_sales")
leap_yr_sales.show()
#leap_yr_sales.count()

+-----+------------+
|Store|Weekly_Sales|
+-----+------------+
|    5|     5253.56|
|   33|      5695.0|
|   44|     6384.96|
|    3|     6621.76|
|   36|     7007.63|
|   16|     7535.77|
|   29|     7851.88|
|   38|     8142.95|
|   15|     8345.39|
|    7|     8605.01|
|   30|     8725.93|
|    9|     8905.66|
|   25|     9986.63|
|   37|    10027.84|
|   21|     10424.5|
|   45|    11264.59|
|   42|    11361.73|
|   35|    12252.34|
|    8|    13192.31|
|   43|    13334.27|
+-----+------------+
only showing top 20 rows



Sales Prediction with Unemployment Factor:

In [17]:
Retail_frame.createOrReplaceTempView("sales_pred")
sales_pred_unemp_factr =  spark.sql("select Dept,round(sum(cast(Weekly_Sales as decimal)),2) as Weekly_Sales from sales_pred where Unemployment >8 group by Dept order by Weekly_sales")
sales_pred_unemp_factr.show()
#sales_pred_unemp_factr.count()

+----+------------+
|Dept|Weekly_Sales|
+----+------------+
|  47|       -5504|
|  43|          10|
|  39|         109|
|  78|         999|
|  77|       13237|
|  51|       19428|
|  45|       19554|
|  99|       91414|
|  54|      258833|
|  48|      500884|
|  60|      913425|
|  50|     1506755|
|  28|     1822382|
|  59|     2105703|
|  19|     2714019|
|  37|     2820114|
|  27|     4077452|
|  41|     4632271|
|  36|     4867078|
|  52|     5062862|
+----+------------+
only showing top 20 rows



Monthly Sales Aggregation:

In [18]:
Retail_frame.createOrReplaceTempView("dept_sales_mnth")
dpt_sales_mnth = spark.sql("select dept, sum(cast(Weekly_Sales as decimal)) as sales from dept_sales_mnth group by dept order by dept")
dpt_sales_mnth.show()
#dpt_sales_mnth.count()

+----+---------+
|dept|    sales|
+----+---------+
|   1|123638784|
|   2|280611207|
|   3| 75892451|
|   4|167146819|
|   5|135607428|
|   6| 28420715|
|   7|155477564|
|   8|194280841|
|   9|128393278|
|  10|117897400|
|  11| 93329301|
|  12| 26760174|
|  13|197321600|
|  14| 95694731|
|  16| 91670739|
|  17| 65319843|
|  18| 36897517|
|  19|  6816203|
|  20| 33349669|
|  21| 32064710|
+----+---------+
only showing top 20 rows



Weekly High Sales Store Identification:

In [19]:
Retail_frame.createOrReplaceTempView("week_wise_sales")
week_wise_sales = spark.sql("select store,Date_formatted as Week, sum(cast(Weekly_Sales as decimal)) as sales from week_wise_sales group by store,Week order by sales desc")
week_wise_sales.show()
#week_wise_sales.count()

+-----+----------+-------+
|store|      Week|  sales|
+-----+----------+-------+
|   14|2010-12-24|3818690|
|   20|2010-12-24|3766689|
|   10|2010-12-24|3749057|
|    4|2011-12-23|3676387|
|   13|2010-12-24|3595904|
|   13|2011-12-23|3556768|
|   20|2011-12-23|3555374|
|    4|2010-12-24|3526716|
|   10|2011-12-23|3487991|
|    2|2010-12-24|3436013|
|   14|2011-12-23|3369067|
|    2|2011-12-23|3224373|
|   27|2010-12-24|3078164|
|    4|2011-11-25|3004702|
|   10|2011-11-25|2950199|
|   10|2010-11-26|2939947|
|   14|2010-11-26|2921711|
|   20|2011-11-25|2906232|
|   13|2011-11-25|2864172|
|   20|2010-12-17|2819192|
+-----+----------+-------+
only showing top 20 rows



Department Performance Analysis:

In [20]:
Retail_frame.createOrReplaceTempView("weekly_deptwise_sales")
weekly_deptwise_sales = spark.sql("select store, dept,round(avg(cast(Weekly_Sales as decimal)),2) as Average_sales_across_weeks from weekly_deptwise_sales group by dept, store  order by store, dept")
weekly_deptwise_sales.show()
#weekly_deptwise_sales.count()

+-----+----+--------------------------+
|store|dept|Average_sales_across_weeks|
+-----+----+--------------------------+
|    1|   1|                  22513.31|
|    1|   2|                  46102.10|
|    1|   3|                  13150.51|
|    1|   4|                  36964.16|
|    1|   5|                  24257.93|
|    1|   6|                   4801.79|
|    1|   7|                  24566.45|
|    1|   8|                  35718.28|
|    1|   9|                  28062.06|
|    1|  10|                  31033.41|
|    1|  11|                  24919.27|
|    1|  12|                  10566.56|
|    1|  13|                  38692.84|
|    1|  14|                  15268.55|
|    1|  16|                  24151.08|
|    1|  17|                   9196.57|
|    1|  18|                   7765.32|
|    1|  19|                   1259.02|
|    1|  20|                   4091.57|
|    1|  21|                   7808.43|
+-----+----+--------------------------+
only showing top 20 rows



Fuel Price Analysis:

In [21]:
Retail_frame.createOrReplaceTempView("week_wise_fuel_price")
week_wise_fuel_price = spark.sql("select store, Date_formatted, min(Fuel_Price) as fuel_price from week_wise_fuel_price group by store,Date_formatted order by fuel_price asc")
week_wise_fuel_price.show()
#week_wise_fuel_price.count()

+-----+--------------+----------+
|store|Date_formatted|fuel_price|
+-----+--------------+----------+
|   36|    2010-02-19|     2.472|
|   36|    2010-09-10|     2.513|
|   30|    2010-02-19|     2.514|
|    1|    2010-02-19|     2.514|
|   31|    2010-02-19|     2.514|
|    8|    2010-02-19|     2.514|
|   11|    2010-02-19|     2.514|
|   21|    2010-02-19|     2.514|
|    9|    2010-02-19|     2.514|
|   43|    2010-02-19|     2.514|
|    6|    2010-02-19|     2.514|
|    2|    2010-02-19|     2.514|
|    3|    2010-02-19|     2.514|
|    5|    2010-02-19|     2.514|
|   39|    2010-02-19|     2.514|
|   37|    2010-02-19|     2.514|
|   36|    2010-02-26|      2.52|
|   36|    2010-09-03|     2.533|
|   36|    2010-02-12|     2.539|
|    4|    2010-02-19|      2.54|
+-----+--------------+----------+
only showing top 20 rows



Yearly Store Performance Analysis:

In [22]:
Retail_frame.createOrReplaceTempView("yearly_store_analysis")
yearly_store_analysis = spark.sql("select store, year(Date_formatted) as year, round(avg(cast(Weekly_Sales as decimal)),2) as Avg_sales,round(sum(cast(Weekly_Sales as decimal)),2) as Total_sales  from yearly_store_analysis group by store, year order by store, year")
yearly_store_analysis.show()
#yearly_store_analysis.count()

+-----+----+---------+-----------+
|store|year|Avg_sales|Total_sales|
+-----+----+---------+-----------+
|    1|2010| 21283.44|   73278876|
|    1|2011| 21718.18|   80921941|
|    1|2012| 22179.54|   68202070|
|    2|2010| 27794.02|   95277908|
|    2|2011| 26443.52|   98607892|
|    2|2012| 26451.39|   81496723|
|    3|2010|  6178.48|   18745497|
|    3|2011|  6346.62|   20816913|
|    3|2012|  6621.77|   18024459|
|    4|2010| 27709.40|   95680567|
|    4|2011| 29831.45|  111092315|
|    4|2012| 29974.55|   92771246|
|    5|2010|  4951.96|   14836084|
|    5|2011|  4982.11|   16470868|
|    5|2012|  5253.57|   14168867|
|    6|2010| 22554.95|   76912372|
|    6|2011| 21606.86|   80528769|
|    6|2012| 21572.90|   66315083|
|    7|2010|  7788.04|   25568123|
|    7|2011|  8683.85|   30662682|
+-----+----+---------+-----------+
only showing top 20 rows



Weekly Performance Analysis with/without Offers:

In [45]:
Retail_frame.createOrReplaceTempView("offer_data")
offer_data = spark.sql("select Store,Sales_Date,(Total_discount + Sales) as Discounted_sales, Sales  from (select store,Date_formatted as Sales_Date, sum(MarkDown1+MarkDown2+MarkDown3+MarkDown4+MarkDown5) as Total_discount, sum(weekly_sales)as Sales from offer_data group by store, Date_formatted order by store desc, Sales_date desc)").show()

+-----+----------+------------------+-----------------+
|Store|Sales_Date|  Discounted_sales|            Sales|
+-----+----------+------------------+-----------------+
|   45|2012-10-26|        1111847.85|        760281.43|
|   45|2012-10-19|         980965.25|        718125.53|
|   45|2012-10-12|1167030.3399999999|734464.3599999999|
|   45|2012-10-05|        1390267.07|733455.0700000001|
|   45|2012-09-28|1338062.6100000003|713173.9500000002|
|   45|2012-09-21|         2039937.7|         723086.2|
|   45|2012-09-14|2089337.6800000006|702238.2700000001|
|   45|2012-09-07|1801536.8099999996|        766512.66|
|   45|2012-08-31|3095303.4299999997|734297.8699999999|
|   45|2012-08-24|1778615.7999999998|        718232.26|
|   45|2012-08-17|1322358.3900000001|722496.9299999999|
|   45|2012-08-10|        2260427.48|733037.3200000001|
|   45|2012-08-03|3404418.5300000003|725729.5100000001|
|   45|2012-07-27|1935127.0900000008|711671.5800000001|
|   45|2012-07-20|1491897.6399999997|        737